In [9]:
import numpy as np
import pandas as pd
from datetime import timedelta
import math

from dateutil.relativedelta import *
from datetime import date
from datetime import datetime
import fprocessamento as fprocess


In [10]:
#ISO-8859-1
nomeArquivoOrigemAnoMes = 'datasets/origem/MPS/ACT02.csv'
nomeArquivoResultadoAnoMesTotal = 'datasets/result/dados_anoMes_MPS_t.csv'
nomeArquivoResultadoAnoMes = 'datasets/result/dados_anoMes_MPS.csv'
nomeArquivoResultadoAnoTotal = 'datasets/result/dados_anoTotal_MPS_t.csv'

columns_types = {'ano': str, 'mes': str, 'motivo"': str, 'qtde': str, 'anoMes': str, 'situacao': str}

nomes  = ['ano', 'mes', 'motivo', 'qtde', 'anoMes', 'situacao']

dados_originais = pd.read_csv(nomeArquivoOrigemAnoMes, sep = ',', header=0, names=nomes, 
                              dtype=columns_types, encoding='ISO-8859-1')

df_original = pd.DataFrame(data = dados_originais, columns=nomes)
df_original.shape[0] #1320

1320

In [11]:
df_original.head()

,ano,mes,motivo,qtde,anoMes,situacao
0,1997,Janeiro,Típico-Com Cat,27116,199701,Com Cat Registrada
1,1997,Fevereiro,Típico-Com Cat,24612,199702,Com Cat Registrada
2,1997,Março,Típico-Com Cat,28273,199703,Com Cat Registrada
3,1997,Abril,Típico-Com Cat,28374,199704,Com Cat Registrada
4,1997,Maio,Típico-Com Cat,28997,199705,Com Cat Registrada


In [12]:
df_original.drop(['motivo'], axis=1, inplace = True)
fprocess.formatarStr(df_original, ['ano', 'mes', 'qtde', 'anoMes', 'situacao'])

selecaoRegistrados = df_original['situacao'] == 'Com Cat Registrada'
df_original = df_original[selecaoRegistrados].copy()
df_original.shape[0] #792

792

In [13]:
df_original.drop(['situacao'], axis=1, inplace = True)
df_original.drop(['mes'], axis=1, inplace = True)

df_original.dropna(subset = ['qtde'], inplace = True)
df_original.dropna(subset = ['anoMes'], inplace = True)

#excluir os registros cujo anoMes foi preenchido com '{ñ class}'
selecaoIgnorado = df_original['anoMes'] != '{ñ class}'
df_anoMes = df_original[selecaoIgnorado].copy()

df_anoMes['ano'] = df_anoMes['ano'].apply(pd.to_numeric)
df_anoMes['anoMes'] = df_anoMes['anoMes'].apply(pd.to_numeric)
df_anoMes['qtde'] = df_anoMes['qtde'].apply(pd.to_numeric)

selecao2016 = df_anoMes['ano'] >= 2016
df_anoMes = df_anoMes[selecao2016].copy()

df_anoMes.shape[0] #72

72

In [14]:
df_anoMes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 1086 to 1201
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ano     72 non-null     int64
 1   qtde    72 non-null     int64
 2   anoMes  72 non-null     int64
dtypes: int64(3)
memory usage: 2.2 KB


In [15]:
df_anoMes.index = range(df_anoMes.shape[0]) #0:5511
df_anoMes.head(20)

,ano,qtde,anoMes
0,2016,28592,201601
1,2016,1085,201601
2,2016,7967,201601
3,2016,28823,201602
4,2016,1205,201602
5,2016,8202,201602
6,2016,32233,201603
7,2016,1373,201603
8,2016,9431,201603
9,2016,29593,201604


In [16]:
df_result = pd.DataFrame(df_anoMes.groupby(['ano', 'anoMes'], as_index=False)['qtde'].sum())
df_result.head(30)

,ano,anoMes,qtde
0,2016,201601,37644
1,2016,201602,38230
2,2016,201603,43037
3,2016,201604,40084
4,2016,201605,40407
5,2016,201606,42128
6,2016,201607,40629
7,2016,201608,43736
8,2016,201609,39781
9,2016,201610,38935


In [17]:
df_result.shape[0] #24

24

In [18]:
df_resultAno = df_result.copy()
df_resultAnoTotal = pd.DataFrame(df_resultAno.groupby(['ano'], as_index=False)['qtde'].sum())
df_resultAnoTotal.head(30)

,ano,qtde
0,2016,478039
1,2017,453839


In [12]:
df_result.to_csv(nomeArquivoResultadoAnoMesTotal, sep = ';', encoding='UTF-8')
df_resultAnoTotal.to_csv(nomeArquivoResultadoAnoTotal, sep = ';', encoding='UTF-8')
